# Initial exploration of each data file
Pandas makes importing data from files easy. But sometimes the file contents are poorly formatted or can hold hidden surprises. Make sure that the data - and data types - are what you expect them to be before starting your analysis.

In [1]:
import pandas as pd
import numpy as np
import os
from os.path import join

cwd = os.getcwd()
data_path = join(cwd, '..', '..', 'data')

I sometimes find it helpful to change the Pandas viewing options for max rows and max columns

In [2]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

### Don't write absolute paths
An absolute path is something like `/Users/Home/Documents/GitHub/python-data-analysis-class/data/epa_emissions_2016.txt`. Or in Windows it might be `C:\Users\gschivley\Documents\GitHub\python-data-analysis-class\data\epa_emissions_2016.txt`.

Use relative paths and Python built-in tools to write paths.

In [3]:
print(cwd)
print(data_path)

/Users/Home/Documents/GitHub/python-data-analysis-class/notebooks/Pandas
/Users/Home/Documents/GitHub/python-data-analysis-class/notebooks/Pandas/../../data


## Define function to clean column names

In [15]:
def clean_columns(columns):
    'Remove special characters and convert to snake case'
    clean = (columns.str.lower()
                    .str.replace('[^0-9a-zA-Z\-]+', ' ')
                    .str.replace('-', '')
                    .str.strip()
                    .str.replace(' ', '_'))
    return clean

## Set file paths

In [4]:
# Paths to each of the data files (epa emissions and eia generation)

epa_path = join(data_path, 'external', 'epa_emissions_2016.txt')
gen_path = join(data_path, 'external', 'EIA923_Schedules_2_3_4_5_M_12_2016_Final_Revision.xlsx')

## Load EPA emissions data
Lets load the file and see what needs to be done to make sure the data is in good shape and accessible.

In [5]:
epa = pd.read_csv(epa_path)

It looks like the header column is not well aligned with the data. States are showing up as the index and the first column is labeled **State**.

In [6]:
epa.head()

,State,Facility Name,Facility ID (ORISPL),Month,Year,Gross Load (MW-h),SO2 (tons),NOx (tons),CO2 (short tons),Heat Input (MMBtu)
AL,AMEA Sylacauga Plant,56018,1,2016,4534.0,0.014,2.229,3101.8,52585.8,NaN
AL,AMEA Sylacauga Plant,56018,2,2016,792.0,0.002,0.361,542.0,9186.0,NaN
AL,AMEA Sylacauga Plant,56018,3,2016,1498.0,0.005,0.677,1024.2,17365.5,NaN
AL,AMEA Sylacauga Plant,56018,4,2016,1405.0,0.005,0.586,884.1,14987.3,NaN
AL,AMEA Sylacauga Plant,56018,5,2016,1791.0,0.006,0.756,1145.1,19412.8,NaN


In [7]:
with open(epa_path) as f:
    head = [next(f) for x in range(5)]

for line in head:
    print(line) 

State, Facility Name, Facility ID (ORISPL), Month, Year, Gross Load (MW-h), SO2 (tons), NOx (tons), CO2 (short tons), Heat Input (MMBtu)

"AL","AMEA Sylacauga Plant","56018","1","2016","4534","0.014","2.229","3101.8","52585.8",

"AL","AMEA Sylacauga Plant","56018","2","2016","792","0.002","0.361","542","9186",

"AL","AMEA Sylacauga Plant","56018","3","2016","1498","0.005","0.677","1024.2","17365.5",

"AL","AMEA Sylacauga Plant","56018","4","2016","1405","0.005","0.586","884.1","14987.3",



In [8]:
epa = pd.read_csv(epa_path, index_col=False)

In [9]:
epa.head()

,State,Facility Name,Facility ID (ORISPL),Month,Year,Gross Load (MW-h),SO2 (tons),NOx (tons),CO2 (short tons),Heat Input (MMBtu)
0,AL,AMEA Sylacauga Plant,56018,1,2016,4534.0,0.014,2.229,3101.8,52585.8
1,AL,AMEA Sylacauga Plant,56018,2,2016,792.0,0.002,0.361,542.0,9186.0
2,AL,AMEA Sylacauga Plant,56018,3,2016,1498.0,0.005,0.677,1024.2,17365.5
3,AL,AMEA Sylacauga Plant,56018,4,2016,1405.0,0.005,0.586,884.1,14987.3
4,AL,AMEA Sylacauga Plant,56018,5,2016,1791.0,0.006,0.756,1145.1,19412.8


In [10]:
epa.tail()

,State,Facility Name,Facility ID (ORISPL),Month,Year,Gross Load (MW-h),SO2 (tons),NOx (tons),CO2 (short tons),Heat Input (MMBtu)
14476,WY,Wyodak,6101,8,2016,279246.0,233.875,347.809,321484.0,3065249.8
14477,WY,Wyodak,6101,9,2016,267691.0,228.710,334.644,309080.6,2946991.1
14478,WY,Wyodak,6101,10,2016,253110.0,211.495,314.882,290879.0,2773451.5
14479,WY,Wyodak,6101,11,2016,249476.0,207.293,306.374,283543.1,2703507.4
14480,WY,Wyodak,6101,12,2016,252072.0,207.199,304.010,282007.9,2688870.4


### Access parts of the dataframe

Look at the column names

In [11]:
epa.columns

Index(['State', ' Facility Name', ' Facility ID (ORISPL)', ' Month', ' Year',
       ' Gross Load (MW-h)', ' SO2 (tons)', ' NOx (tons)', ' CO2 (short tons)',
       ' Heat Input (MMBtu)'],
      dtype='object')

Notice that most of the columns have a leading space? We need to strip out those leading spaces and it might be nice to do some extra formatting.

In [17]:
epa.columns = clean_columns(epa.columns)
epa.columns

Index(['state', 'facility_name', 'facility_id_orispl', 'month', 'year',
       'gross_load_mwh', 'so2_tons', 'nox_tons', 'co2_short_tons',
       'heat_input_mmbtu'],
      dtype='object')

### Data types of each column
Numeric columns will either be `int` or `float`. If a column is of type `object` it is either all strings or a mix of types. Watch out for columns that should be numeric but should up as `object`.

In [17]:
epa.dtypes

state                  object
facility_name          object
facility_id_orispl      int64
month                   int64
year                    int64
gross_load_mwh        float64
so2_tons              float64
nox_tons              float64
co2_short_tons        float64
heat_input_mmbtu      float64
dtype: object

## Basic statistics of the data

In [18]:
epa.describe()

,facility_id_orispl,month,year,gross_load_mwh,so2_tons,nox_tons,co2_short_tons,heat_input_mmbtu
count,14481.000000,14481.000000,14481.0,1.226200e+04,12465.000000,12700.000000,1.202200e+04,1.272200e+04
mean,22422.507769,6.490988,2016.0,1.974383e+05,117.769365,91.650835,1.582862e+05,1.743909e+06
std,24871.051651,3.442153,0.0,2.926614e+05,369.451131,214.962978,2.678391e+05,2.650419e+06
min,3.000000,1.000000,2016.0,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00
25%,2399.000000,4.000000,2016.0,4.474020e+03,0.028000,1.553750,4.555913e+03,5.153396e+04
50%,7145.000000,6.000000,2016.0,5.686740e+04,0.356000,9.534000,4.536591e+04,5.493512e+05
75%,55238.000000,9.000000,2016.0,2.890370e+05,16.868000,55.851500,1.799689e+05,2.454411e+06
max,70454.000000,12.000000,2016.0,2.093063e+06,5165.046000,2394.967000,2.341848e+06,2.232883e+07


Index into a dataframe using `.loc` or `.iloc` with square brackets and row,column notation

In [19]:
epa.iloc[0:5, :4]

,state,facility_name,facility_id_orispl,month
0,AL,AMEA Sylacauga Plant,56018,1
1,AL,AMEA Sylacauga Plant,56018,2
2,AL,AMEA Sylacauga Plant,56018,3
3,AL,AMEA Sylacauga Plant,56018,4
4,AL,AMEA Sylacauga Plant,56018,5


# Load generation data

In [18]:
generation = pd.read_excel(gen_path, header=5)

In [19]:
generation.head()

,Plant Id,Combined Heat And Power Plant,Nuclear Unit Id,Plant Name,Operator Name,Operator Id,Plant State,Census Region,NERC Region,Reserved,NAICS Code,EIA Sector Number,Sector Name,Reported Prime Mover,Reported Fuel Type Code,AER Fuel Type Code,Reserved.1,Reserved.2,Physical Unit Label,Quantity January,Quantity February,Quantity March,Quantity April,Quantity May,Quantity June,Quantity July,Quantity August,Quantity September,Quantity October,Quantity November,Quantity December,Elec_Quantity January,Elec_Quantity February,Elec_Quantity March,Elec_Quantity April,Elec_Quantity May,Elec_Quantity June,Elec_Quantity July,Elec_Quantity August,Elec_Quantity September,Elec_Quantity October,Elec_Quantity November,Elec_Quantity December,MMBtuPer_Unit January,MMBtuPer_Unit February,MMBtuPer_Unit March,MMBtuPer_Unit April,MMBtuPer_Unit May,MMBtuPer_Unit June,MMBtuPer_Unit July,MMBtuPer_Unit August,MMBtuPer_Unit September,MMBtuPer_Unit October,MMBtuPer_Unit November,MMBtuPer_Unit December,Tot_MMBtu January,Tot_MMBtu February,Tot_MMBtu March,Tot_MMBtu April,Tot_MMBtu May,Tot_MMBtu June,Tot_MMBtu July,Tot_MMBtu August,Tot_MMBtu September,Tot_MMBtu October,Tot_MMBtu November,Tot_MMBtu December,Elec_MMBtu January,Elec_MMBtu February,Elec_MMBtu March,Elec_MMBtu April,Elec_MMBtu May,Elec_MMBtu June,Elec_MMBtu July,Elec_MMBtu August,Elec_MMBtu September,Elec_MMBtu October,Elec_MMBtu November,Elec_MMBtu December,Netgen January,Netgen February,Netgen March,Netgen April,Netgen May,Netgen June,Netgen July,Netgen August,Netgen September,Netgen October,Netgen November,Netgen December,Total Fuel Consumption Quantity,Electric Fuel Consumption Quantity,Total Fuel Consumption MMBtu,Elec Fuel Consumption MMBtu,Net Generation (Megawatthours),YEAR
0,2,N,NaN,Bankhead Dam,Alabama Power Co,195,AL,ESC,SERC,NaN,22,1,Electric Utility,HY,WAT,HYC,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-77.112,-69.679,-48.374,-24.341,-11.476,-11.441,-11.593,-17.407,-11.566,-10.832,-10.27,-20.909,0,0,0,0,-325.000,2016
1,3,N,NaN,Barry,Alabama Power Co,195,AL,ESC,SERC,NaN,22,1,Electric Utility,CA,NG,NG,NaN,NaN,mcf,57253,27242,86461,54707,64489,90182,120407,115553,109091,67504,64673,20895,57253,27242,86461,54707,64489,90182,120407,115553,109091,67504,64673,20895,1.017,1.019,1.02,1.018,1.017,1.017,1.014,1.022,1.024,1.021,1.019,1.024,58226,27760,88190,55692,65585,91715,122093,118095,111709,68922,65902,21396,58226,27760,88190,55692,65585,91715,122093,118095,111709,68922,65902,21396,268797,251423,214816,261564,223662,206449,259415,229111,251839,219263,266927,265011,878457,878457,895285,895285,2918277.000,2016
2,3,N,NaN,Barry,Alabama Power Co,195,AL,ESC,SERC,NaN,22,1,Electric Utility,CT,NG,NG,NaN,NaN,mcf,5248798,4781915,4076291,5053433,4355654,4066516,5055348,4780696,4824758,4159500,5142640,5148943,5248798,4781915,4076291,5053433,4355654,4066516,5055348,4780696,4824758,4159500,5142640,5148943,1.017,1.019,1.02,1.018,1.017,1.017,1.014,1.022,1.024,1.021,1.019,1.024,5338028,4872771,4157817,5144395,4429700,4135647,5126123,4885871,4940552,4246850,5240350,5272518,5338028,4872771,4157817,5144395,4429700,4135647,5126123,4885871,4940552,4246850,5240350,5272518,511773,457142,396211,490439,421779,388783,477329,477474,463646,401506,495929,501072,56694492,56694492,57790622,57790622,5483083.000,2016
3,3,N,NaN,Barry,Alabama Power Co,195,AL,ESC,SERC,NaN,22,1,Electric Utility,ST,BIT,COL,NaN,NaN,short tons,159951,129081,137723,19982,178956,241627,260989,273207,228027,144161,169610,100688,159951,129081,137723,19982,178956,241627,260989,273207,228027,144161,169610,100688,20.589,20.736,20.851,20.838,21.425,21.299,21.405,21.317,20.995,20.742,21.042,21.278,3293231,2676624,2871662,416385,3834132,5146413,5586470,5823954,4787427,2990187,3568934,2142439,3293231,2676624,2871662,416385,3834132,5146413,5586470,5823954,4787427,2990187,3568934,2142439,329513,259648,291745,35269.8,376727,510204,559747,583589,473794,286736,363877,207463,2

In [20]:
generation.columns

Index(['Plant Id', 'Combined Heat And\nPower Plant', 'Nuclear Unit Id',
       'Plant Name', 'Operator Name', 'Operator Id', 'Plant State',
       'Census Region', 'NERC Region', 'Reserved', 'NAICS Code',
       'EIA Sector Number', 'Sector Name', 'Reported\nPrime Mover',
       'Reported\nFuel Type Code', 'AER\nFuel Type Code', 'Reserved.1',
       'Reserved.2', 'Physical\nUnit Label', 'Quantity\nJanuary',
       'Quantity\nFebruary', 'Quantity\nMarch', 'Quantity\nApril',
       'Quantity\nMay', 'Quantity\nJune', 'Quantity\nJuly', 'Quantity\nAugust',
       'Quantity\nSeptember', 'Quantity\nOctober', 'Quantity\nNovember',
       'Quantity\nDecember', 'Elec_Quantity\nJanuary',
       'Elec_Quantity\nFebruary', 'Elec_Quantity\nMarch',
       'Elec_Quantity\nApril', 'Elec_Quantity\nMay', 'Elec_Quantity\nJune',
       'Elec_Quantity\nJuly', 'Elec_Quantity\nAugust',
       'Elec_Quantity\nSeptember', 'Elec_Quantity\nOctober',
       'Elec_Quantity\nNovember', 'Elec_Quantity\nDecember',
   

In [21]:
generation.dtypes

Plant Id                                 int64
Combined Heat And\nPower Plant          object
Nuclear Unit Id                        float64
Plant Name                              object
Operator Name                           object
Operator Id                              int64
Plant State                             object
Census Region                           object
NERC Region                             object
Reserved                               float64
NAICS Code                               int64
EIA Sector Number                        int64
Sector Name                             object
Reported\nPrime Mover                   object
Reported\nFuel Type Code                object
AER\nFuel Type Code                     object
Reserved.1                             float64
Reserved.2                             float64
Physical\nUnit Label                    object
Quantity\nJanuary                       object
Quantity\nFebruary                      object
Quantity\nMar

In [22]:
generation.loc[~generation['Netgen\nJanuary'].map(lambda x: isinstance(x, (int, float))), 'Netgen\nJanuary'].unique()

array(['.'], dtype=object)

### Melt generation data to tidy format and groupby facility
`melt` is a function that takes wide-form data and transforms it into tidy data.

In [23]:
# Reload with periods as na values
generation = pd.read_excel(gen_path, header=5, na_values='.')

In [25]:
generation.columns = clean_columns(generation.columns)
generation.columns

Index(['plant_id', 'combined_heat_and_power_plant', 'nuclear_unit_id',
       'plant_name', 'operator_name', 'operator_id', 'plant_state',
       'census_region', 'nerc_region', 'reserved', 'naics_code',
       'eia_sector_number', 'sector_name', 'reported_prime_mover',
       'reported_fuel_type_code', 'aer_fuel_type_code', 'reserved_1',
       'reserved_2', 'physical_unit_label', 'quantity_january',
       'quantity_february', 'quantity_march', 'quantity_april', 'quantity_may',
       'quantity_june', 'quantity_july', 'quantity_august',
       'quantity_september', 'quantity_october', 'quantity_november',
       'quantity_december', 'elec_quantity_january', 'elec_quantity_february',
       'elec_quantity_march', 'elec_quantity_april', 'elec_quantity_may',
       'elec_quantity_june', 'elec_quantity_july', 'elec_quantity_august',
       'elec_quantity_september', 'elec_quantity_october',
       'elec_quantity_november', 'elec_quantity_december',
       'mmbtuper_unit_january', 'mmbtup

#### Identify columns with data values to keep
The `value_cols` parameter in `melt` identifies columns with values to keep. These are quantitative values, not categorical information. The column names become categorical information in a new column.

In [26]:
# Use a list comprehension to identify columns with 'netgen' in their name
value_cols = [col for col in generation.columns if 'netgen' in col]

#### Melt and group by `plant_id` and the new `month` column
This can be done in multiple steps. I'm combining it into a single step here by *chaining* methods.

In [29]:
pd.melt(generation, id_vars=['plant_id'],
        value_vars=value_cols, value_name='net_gen',
        var_name='month').head()

,plant_id,month,net_gen
0,2,netgen_january,-77.112
1,3,netgen_january,268797.000
2,3,netgen_january,511773.000
3,3,netgen_january,329513.400
4,3,netgen_january,13195.605


In [27]:
df = (pd.melt(generation, id_vars=['plant_id'],
              value_vars=value_cols, value_name='net_gen',
              var_name='month')
        .groupby(['plant_id', 'month'], as_index=False)
        .sum())

In [28]:
df.head()

,plant_id,month,net_gen
0,2,netgen_april,-24.341
1,2,netgen_august,-17.407
2,2,netgen_december,-20.909
3,2,netgen_february,-69.679
4,2,netgen_january,-77.112


In [30]:
# The netgen column names became values in a month column
df['month'] = df.month.str.replace('netgen_', '')

In [31]:
df.head()

,plant_id,month,net_gen
0,2,april,-24.341
1,2,august,-17.407
2,2,december,-20.909
3,2,february,-69.679
4,2,january,-77.112


### Convert month columns to integer values
The EPA emissions data also has a `month` column but the values are integers. We'll use a built-in list of months from the `calendar` package to create a maping of names to integers.

In [32]:
epa.month.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [33]:
import calendar

In [34]:
# This is an object in memory
calendar.month_name

In [35]:
# View values by converting to a list
list(calendar.month_name)

['',
 'January',
 'February',
 'March',
 'April',
 'May',
 'June',
 'July',
 'August',
 'September',
 'October',
 'November',
 'December']

In [36]:
# Will need to capitalize the month names to match the list
df.month.str.capitalize().head()

0       April
1      August
2    December
3    February
4     January
Name: month, dtype: object

In [37]:
# Create a map of month names to numbers
month_map = {month: idx for idx, month in enumerate(calendar.month_name)}
month_map

{'': 0,
 'January': 1,
 'February': 2,
 'March': 3,
 'April': 4,
 'May': 5,
 'June': 6,
 'July': 7,
 'August': 8,
 'September': 9,
 'October': 10,
 'November': 11,
 'December': 12}

In [38]:
df['month'] = df.month.str.capitalize().map(month_map)

In [39]:
df.head()

,plant_id,month,net_gen
0,2,4,-24.341
1,2,8,-17.407
2,2,12,-20.909
3,2,2,-69.679
4,2,1,-77.112


In [40]:
df.sort_values(['plant_id', 'month'], inplace=True)

In [41]:
df.month.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [42]:
df.head()

,plant_id,month,net_gen
4,2,1,-77.112
3,2,2,-69.679
7,2,3,-48.374
0,2,4,-24.341
8,2,5,-11.476


### What fuel is used to generate the most electricity at each plant?

- Melt the original data again and groupby by the `plant_id` and `aer_fuel_type_code`
- Group by `plant_id` and apply the `idxmax()` method to just the `net_gen` column to get the index **from the original `df_fuel` dataframe**
- Use the index of locations with max fuels by plant to create a new dataframe with columns of `plant_id` and `aer_fuel_type_code` - but only one listing per `plant_id`
- Merge this dataframe into the monthly generation `df` from above

In [44]:
df_fuel = (pd.melt(generation, id_vars=['plant_id', 'aer_fuel_type_code'],
              value_vars=value_cols, value_name='net_gen',
              var_name='month')
             .groupby(['plant_id', 'aer_fuel_type_code'], as_index=False)
             .sum())

In [45]:
df_fuel.head()

,plant_id,aer_fuel_type_code,net_gen
0,2,HYC,-325.000
1,3,COL,4278313.069
2,3,NG,8492577.936
3,4,HYC,468960.000
4,7,COL,0.000


In [46]:
# View one of the grouped dataframes
list(df_fuel.groupby('plant_id'))[1][-1]

,plant_id,aer_fuel_type_code,net_gen
1,3,COL,4278313.069
2,3,NG,8492577.936


In [54]:
# The groupby parameter "as_index" is True by default, 
# so this code finds the original row index value with the max generation
# by plant and makes it the value in a new series where plant_id is the index
max_fuel_index = df_fuel.groupby('plant_id')['net_gen'].idxmax()
max_fuel_index.head()

plant_id
2    0
3    2
4    3
7    5
8    7
Name: net_gen, dtype: int64

In [58]:
# There are several ways we could get to the final desired result.
# I'm going to use the max_fuel_index values to create a new dataframe
# with the fuel type code and plant_id from df_fuel
max_fuel = df_fuel.loc[max_fuel_index, ['aer_fuel_type_code', 'plant_id']]
max_fuel.columns = ['primary_gen_fuel', 'plant_id']
max_fuel.head()

,primary_gen_fuel,plant_id
0,HYC,2
2,NG,3
3,HYC,4
5,NG,7
7,COL,8


In [59]:
# Finally, merge the new max_fuel dataframe back into df
df.merge(max_fuel, on='plant_id')

,plant_id,month,net_gen,primary_gen_fuel
0,2,1,-77.112,HYC
1,2,2,-69.679,HYC
2,2,3,-48.374,HYC
3,2,4,-24.341,HYC
4,2,5,-11.476,HYC
5,2,6,-11.441,HYC
6,2,7,-11.593,HYC
7,2,8,-17.407,HYC
8,2,9,-11.566,HYC
9,2,10,-10.832,HYC
